In [6]:
import warnings
warnings.filterwarnings('ignore')


import tensorflow.compat.v1 as tf

from tensorflow.compat.v1.keras.datasets.mnist import load_data
tf.logging.set_verbosity(tf.logging.ERROR)

import matplotlib.pyplot as plt

In [10]:
(x_train, y_train), (x_test, y_test) = load_data()

11493376/11490434 [==============================] - 4s 0us/step


In [22]:
#number of neurons in input layer
num_input = 784  

#number of neurons in hidden layer 1
num_hidden1 = 512  

#number of neurons in hidden layer 2
num_hidden2 = 256  

#number of neurons in hidden layer 3
num_hidden_3 = 128  

#number of neurons in output layer
num_output = 10

In [24]:
tf.disable_eager_execution()
with tf.name_scope('input'):
    X = tf.placeholder("float", [None, num_input])

with tf.name_scope('output'):
    Y = tf.placeholder("float", [None, num_output])


Since we have a 4 layer network, we have 4 weights and 4 baises. We initialize our weights by drawing values from the truncated normal distribution with a standard deviation of 0.1.

Remember, the dimensions of the weights matrix should be a number of neurons in the previous layer x number of neurons in the current layer. For instance, the dimension of weight matrix w3 should be the number of neurons in the hidden layer 2 x number of neurons in hidden layer 3.

We often define all the weights in a dictionary as given below:

In [25]:
with tf.name_scope('weights'):
    
        weights = {
        'w1': tf.Variable(tf.truncated_normal([num_input, num_hidden1], stddev=0.1),name='weight_1'),
        'w2': tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2], stddev=0.1),name='weight_2'),
        'w3': tf.Variable(tf.truncated_normal([num_hidden2, num_hidden_3], stddev=0.1),name='weight_3'),
        'out': tf.Variable(tf.truncated_normal([num_hidden_3, num_output], stddev=0.1),name='weight_4'),
    }

The dimension of bias should be a number of neurons in the current layer. For instance, the dimension of bias b2 is the number of neurons in the hidden layer 2. We set the bias value as constant 0.1 in all layers:

In [26]:

with tf.name_scope('biases'):

    biases = {
        'b1': tf.Variable(tf.constant(0.1, shape=[num_hidden1]),name='bias_1'),
        'b2': tf.Variable(tf.constant(0.1, shape=[num_hidden2]),name='bias_2'),
        'b3': tf.Variable(tf.constant(0.1, shape=[num_hidden_3]),name='bias_3'),
        'out': tf.Variable(tf.constant(0.1, shape=[num_output]),name='bias_4')
    }

## Forward Propagation

Now, we define the forward propagation operation. We use relu activations in all layers and in the last layer we use sigmoid activation as defined below:

In [27]:
with tf.name_scope('Model'):
    
    with tf.name_scope('layer1'):
        layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['w1']), biases['b1']) )   
    
    with tf.name_scope('layer2'):
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
        
    with tf.name_scope('layer3'):
        layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
        
    with tf.name_scope('output_layer'):
         y_hat = tf.nn.sigmoid(tf.matmul(layer_3, weights['out']) + biases['out'])


## Compute Loss and Backpropagate
Next, we define our loss function. We use softmax cross-entropy as our loss function. Tensorflow provides tf.nn.softmax_cross_entropy_with_logits() function for computing the softmax cross entropy loss. It takes the two parameters as inputs logits and labels.

logits implies the logits predicted by our network. That is, y_hat

labels imply the actual labels. That is, true labels y

We take mean of the loss using tf.reduce_mean()

In [28]:
with tf.name_scope('Loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat,labels=Y))

Now, we need to minimize the loss using backpropagation. Don't worry! We don't have to calculate derivatives of all the weights manually. Instead, we can use tensorflow's optimizer. In this section, we use Adam optimizer. It is a variant of gradient descent optimization technique we learned in the previous chapter. In the next chapter, we will dive into detail and see how exactly all the Adam and several other optimizers work. For now, let's say we use Adam optimizer as our backpropagation algorithm,

tf.train.AdamOptimizer() requires the learning rate as input. So we set 1e-4 as the learning rate and we minimize the loss with minimize() function. It computes the gradients and updates the parameters (weights and biases) of our network.

In [30]:
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

## Compute Accuracy¶
We calculate the accuracy of our model as follows.

y_hat denotes the predicted probability for each class by our model. Since we have 10 classes we will have 10 probabilities. If the probability is high at position 7, then it means that our network predicts the input image as digit 7 with high probability. tf.argmax() returns the index of the largest value. Thus, tf.argmax(y_hat,1) gives the index where the probability is high. Thus, if the probability is high at index 7, then it returns 7
Y denotes the actual labels and it is the one hot encoded values. That is, it consists of zeros everywhere except at the position of the actual image where it consists of 1. For instance, if the input image is 7, then Y has 0 at all indices except at index 7 where it has 1. Thus, tf.argmax(Y,1) returns 7 because that is where we have high value i.e 1.
Thus, tf.armax(y_hat,1) gives the predicted digit and tf.argmax(Y,1) gives us the actual digit.

tf.equal(x, y) takes x and y as inputs and returns the truth value of (x == y) element-wise. Thus, correct_pred = tf.equal(predicted_digit,actual_digit) consists of True where the actual and predicted digits are same and False where the actual and predicted digits are not the same. We convert the boolean values in correct_pred into float using tensorflow's cast operation. That is, tf.cast(correct_pred, tf.float32). After converting into float values, take the average using tf.treduce_mean().

Thus, tf.reduce_mean(tf.cast(correct_pred, tf.float32)) gives us the average correct predictions.

In [31]:
with tf.name_scope('Accuracy'):
    
    predicted_digit = tf.argmax(y_hat, 1)
    actual_digit = tf.argmax(Y, 1)
    
    correct_pred = tf.equal(predicted_digit,actual_digit)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Create Summary¶
We can also visualize how the loss and accuracy of our model change during several iterations in tensorboard. So, we use tf.summary() to get the summary of the variable. Since the loss and accuracy are scalar variables, we use tf.summary.scalar() to store the summary as shown below:

In [32]:
tf.summary.scalar("Accuracy", accuracy)

tf.summary.scalar("Loss", loss)

<tf.Tensor 'Loss_1:0' shape=() dtype=string>

Next, we merge all the summaries we use in our graph using tf.summary.merge_all(). We merge all summaries because when we have many summaries running and storing them would become inefficient, so we merge all the summaries and run them once in our session instead of running multiple times.

In [33]:
merge_summary = tf.summary.merge_all()

## Train the Model¶
Now it is time to train our model. As we learned, first we need to initialize all the variables:

In [34]:
init = tf.global_variables_initializer()

batch_size = 128
num_iterations = 1000

In [35]:

with tf.Session() as sess:

    #run the initializer
    sess.run(init)

    #save the event files
    summary_writer = tf.summary.FileWriter('./graphs', graph=tf.get_default_graph())

    #train for some n number of iterations
    for i in range(num_iterations):
        
        #get batch of data according to batch size
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        #train the network
        sess.run(optimizer, feed_dict={
            X: batch_x, Y: batch_y
            })

        #print loss and accuracy on every 100th iteration
        if i % 100 == 0:
            
            #compute loss, accuracy and summary
            batch_loss, batch_accuracy,summary = sess.run(
                [loss, accuracy, merge_summary], feed_dict={X: batch_x, Y: batch_y}
                )

            #store all the summaries
            summary_writer.add_summary(summary, i)


            print('Iteration: {}, Loss: {}, Accuracy: {}'.format(i,batch_loss,batch_accuracy))

NameError: name 'mnist' is not defined